# Building Network + Data processing :)


## Step 2: Building a Model
### Instantiating a small covnet for controls vs patients - with dropout

In [1]:
from keras import layers
from keras import models
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu',input_shape=(90, 80, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.5))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.5))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.5))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dropout(0.5)) 
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))    

Using TensorFlow backend.


In [2]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 88, 78, 32)        896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 44, 39, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 44, 39, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 42, 37, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 21, 18, 64)        0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 21, 18, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 19, 16, 128)       73856     
__________

### Configuring model for training

In [3]:
from keras import optimizers 
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['acc'])

## Step 3: Data preprocessing 

### - Rescale the pixel values (between 0 and 255) to the [0, 1] interval (as you know, neural networks prefer to deal with small input values)

In [4]:
from keras.preprocessing.image import ImageDataGenerator

train_dir= '/data1/Code/Duaa/Relocated_Data2/Train/'
validation_dir= '/data1/Code/Duaa/Relocated_Data2/Validation/'

#Rescales all images by 1/255
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
         train_dir,                 #Target directory
         target_size = (90, 80),    
         batch_size = 64,
         class_mode = 'binary')     #Because you use binary_crossentropy loss, you need binary labels.
validation_generator = test_datagen.flow_from_directory(
         validation_dir,            #Target directory
         target_size =(90, 80),     
         batch_size = 64,           #Because you use binary_crossentropy loss, you need binary labels.  
         class_mode = 'binary')


# Break loop at some point to stop from having batches run indefinitely
for data_batch, labels_batch in train_generator:
    print('data batch shape:', data_batch.shape)
    print('labels batch shape:', labels_batch.shape)
    break

Found 840538 images belonging to 2 classes.
Found 323455 images belonging to 2 classes.
data batch shape: (64, 90, 80, 3)
labels batch shape: (64,)


#### Fitting the model using a batch generator

In [ ]:
history = model.fit_generator(
train_generator,
steps_per_epoch=13134,
epochs=20,
validation_data=validation_generator,
validation_steps=5053)

Epoch 1/20
   97/13134 [..............................] - ETA: 2:43:23 - loss: 0.6627 - acc: 0.6099

#### Saving the model

In [ ]:
model.save('Controls_and_Patients_droputfull0.5_Batch64_data2.h5')

In [ ]:
import matplotlib.pyplot as plt
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()